## LCEL 

> https://www.youtube.com/watch?v=8aUYzb1aYDU

---

In [16]:
import os
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain_core.output_parsers import StrOutputParser

In [3]:
GROQ_API_KEY = ""
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [5]:
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [6]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='I love programming in French is: "J\'aime programmer."', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 30, 'total_tokens': 46, 'completion_time': 0.024157488, 'prompt_time': 0.003244877, 'queue_time': 0.009945583, 'total_time': 0.027402365}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-55310e50-02c3-40b7-ba4e-64aecdd02aeb-0', usage_metadata={'input_tokens': 30, 'output_tokens': 16, 'total_tokens': 46})

### Simple chains using LCEL

In [10]:
template = "Hi! I am learning {skill}. Can you help point me to top 5 courses for it."
prompt = PromptTemplate(template=template, input_variables=['skill'])
print (prompt)

input_variables=['skill'] input_types={} partial_variables={} template='Hi! I am learning {skill}. Can you help point me to top 5 courses for it.'


In [11]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

/var/folders/9g/tslm33j92p1blxwl8vvjxn1r0000gn/T/ipykernel_18185/4035131671.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)


In [14]:
print (llm_chain.run({"skill": "Video editing"}))

Hello! I'd be happy to help you find some top-notch video editing courses. Here are five courses that I highly recommend:

1. Adobe Premiere Pro Masterclass: Learn Video Editing in Premiere Pro - This course is great for beginners who want to learn Adobe Premiere Pro, one of the most popular video editing software. It covers all the basics and advanced techniques you need to know to create professional-looking videos.
2. Video Editing with Adobe Premiere Pro for Beginners: Start Editing Video Like a Pro - This course is another excellent option for beginners who want to learn Adobe Premiere Pro. It covers the fundamentals of video editing, including cutting, transitions, and color correction.
3. Final Cut Pro X: Video Editing Masterclass - If you're a Mac user, this course is an excellent choice for learning Final Cut Pro X, another popular video editing software. It covers everything from basic editing techniques to advanced color grading and audio mixing.
4. Video Editing in DaVinci 

In [19]:
chain = prompt | llm | StrOutputParser()

print (chain.invoke({"skill": "ML Engineering"}))

Hello! I'm glad to hear that you're interested in machine learning engineering. Here are five top courses that can help you get started:

1. Machine Learning Engineering on Coursera: This course is offered by the University of Washington and covers the entire machine learning engineering pipeline, from data preparation to model deployment. It includes hands-on projects and is a great way to get started with machine learning engineering.
2. Machine Learning on edX: This course is offered by MIT and covers the fundamentals of machine learning, including supervised and unsupervised learning, linear models, and neural networks. It's a great course for those who want to build a strong foundation in machine learning before diving into engineering.
3. Deep Learning Specialization on Coursera: This is a five-course specialization offered by Andrew Ng, a pioneer in the field of deep learning. The courses cover neural networks, deep learning, structuring machine learning projects, convolutional 

### Runnables